# Demonstration of Givens rotation decomposition

In [1]:
import numpy
from openfermion.utils import givens_decomposition
from scipy.linalg import qr

numpy.set_printoptions(precision=3, linewidth=150, suppress=True)

A Slater determinant can be represented by an $m \times n$ matrix with orthonormal rows. Let's obtain such a matrix $Q$ using Scipy's QR decomposition algorithm.

In [2]:
m, n = (3, 6)

# Obtain a random matrix of orthonormal rows
x = numpy.random.randn(n, n)
y = numpy.random.randn(n, n)
A = x + 1j*y
Q, R = qr(A)
Q = Q[:m, :]
print(Q)

[[-0.075+0.15j  -0.123-0.459j -0.081+0.194j  0.726+0.065j  0.077-0.292j -0.268-0.091j]
 [-0.108+0.169j -0.337+0.201j  0.243+0.284j -0.365+0.197j  0.123-0.139j -0.554-0.391j]
 [ 0.022-0.137j -0.543-0.425j -0.153-0.508j -0.164+0.23j  -0.086+0.281j -0.196+0.14j ]]


There exist unitary matrices $V$ and $U$ such that $VQU^\dagger$ is an $m \times n$ diagonal matrix. The matrix $VQ$ represents the same Slater determinant as $Q$ up to an overall phase, but it has zeros in the upper diagonal, which saves some work in the preparation of the Slater determinant. The unitary $U$ can be written in the form

$$U = G_{N_G} \cdots G_2 G_1$$

where the $G_k$ are complex Givens rotations. We can obtain $V$, the $G_k$, and the diagonal entries of $VQU^\dagger$ using the `givens_decomposition` function.

In [3]:
# Get Givens decomposition of U
V, givens_rotations, diagonal = givens_decomposition(Q)
print(V)
print()
print(diagonal)

[[ 0.747+0.047j -0.378+0.229j  0.493+0.049j]
 [ 0.548-0.077j -0.089-0.166j -0.807-0.081j]
 [ 0.366+0.j     0.841-0.248j  0.181+0.253j]]

[-0.677-0.736j -0.105+0.994j -0.181+0.983j]


We didn't print the Givens rotations in the above block because it wouldn't look pretty. The Givens rotations are returned as a list of tuples of tuples. We will now iterate through the tuples of the list, and print the innermost tuple within each tuple as a string.

In [4]:
for parallel_set in givens_rotations:
    print(tuple(["{}, {}, {:.3f}, {:.3f}".format(i, j, theta, phi)
                 for i, j, theta, phi in parallel_set]))

('2, 3, 1.032, -0.493',)
('1, 2, 0.683, 2.427', '3, 4, 0.576, 0.865')
('0, 1, 1.507, -2.707', '2, 3, 0.853, -2.729', '4, 5, 1.524, 3.044')
('1, 2, 1.368, 1.954', '3, 4, 0.914, -3.120')
('2, 3, 1.356, -1.350',)


There are 5 tuples printed, and within each tuple, there are strings of the form 'i, j, theta, phi'. Each such string represents an innermost tuple, and it is a description of a complex Givens rotation of the coordinates $i$ and $j$ by angles $\theta$ and $\varphi$. The $2 \times 2$ matrix corresponding to this rotation is

$$
\begin{pmatrix}
\cos \theta & -e^{i \varphi} \sin \theta \\
\sin \theta & e^{i \varphi} \cos \theta
\end{pmatrix}.
$$

The fact that there are 5 tuples means that the circuit depth to prepare the Slater determinant corresponding to $Q$ (up to a phase) has depth 5. All of the rotations within the tuple can be performed in parallel; this is possible because the indices to be rotated are disjoint. For instance, in the third step, we can perform three rotations simultaneously, on coordinates $(0, 1)$, $(2, 3)$, and $(4, 5)$.

Let's check that $VQ$ has zeros in the upper right corner.

In [5]:
# Check that VQ has zeros in upper right corner
W = V.dot(Q)
print(W)

[[-0.043-0.047j -0.235-0.737j -0.276-0.168j  0.540+0.03j   0.000+0.j     0.000-0.j   ]
 [-0.021+0.2j    0.364+0.183j  0.078+0.47j   0.619-0.15j   0.078-0.394j  0.000-0.j   ]
 [-0.038+0.205j -0.270-0.129j  0.346+0.119j -0.081+0.28j   0.010-0.225j -0.732-0.248j]]


Now let's check the Givens decomposition. For each set of Givens rotations that can be performed in parallel, we construct the matrices corresponding to the Givens rotations and multiply them together. Then, we multiply $W = VQ$ repeatedly on the right by these matrices and check that the correct elements are zeroed out.

In [6]:
# Check the Givens decomposition
def expanded_givens(G, i, j, n):
    expanded_G = numpy.eye(n, dtype=complex)
    expanded_G[([i], [j]), (i, j)] = G
    return expanded_G

U = numpy.eye(n, dtype=complex)
for parallel_set in givens_rotations:
    print("Number of rotations to perform in parallel: {}".format(len(parallel_set)))
    combined_givens = numpy.eye(n)
    for i, j, theta, phi in parallel_set:
        c = numpy.cos(theta)
        s = numpy.sin(theta)
        phase = numpy.exp(1.j * phi)
        G = numpy.array([[c, -phase * s],
                     [s, phase * c]], dtype=complex)
        expanded_G = expanded_givens(G, i, j, n)
        combined_givens = combined_givens.dot(expanded_G)
    W = W.dot(combined_givens.T.conj())
    U = combined_givens.dot(U)
    print(W)
    print()

Number of rotations to perform in parallel: 1
[[-0.043-0.047j -0.235-0.737j -0.538-0.328j -0.000-0.j     0.000+0.j     0.000-0.j   ]
 [-0.021+0.2j    0.364+0.183j -0.489+0.104j  0.383+0.486j  0.078-0.394j  0.000-0.j   ]
 [-0.038+0.205j -0.270-0.129j  0.352-0.118j  0.192+0.209j  0.010-0.225j -0.732-0.248j]]

Number of rotations to perform in parallel: 2
[[-0.043-0.047j -0.303-0.951j  0.000+0.j    -0.000-0.j    -0.000-0.j     0.000-0.j   ]
 [-0.021+0.2j    0.007-0.011j  0.569+0.303j  0.457+0.579j  0.000+0.j     0.000-0.j   ]
 [-0.038+0.205j  0.008-0.011j -0.437-0.191j  0.251+0.259j -0.033-0.015j -0.732-0.248j]]

Number of rotations to perform in parallel: 3
[[-0.677-0.736j  0.000+0.j    -0.000+0.j     0.000+0.j    -0.000-0.j    -0.000-0.j   ]
 [ 0.000-0.j    -0.021+0.2j    0.865+0.46j  -0.000+0.j     0.000-0.j     0.000+0.j   ]
 [ 0.000-0.j    -0.038+0.205j -0.036-0.023j -0.549-0.234j -0.705-0.319j  0.000-0.j   ]]

Number of rotations to perform in parallel: 2
[[-0.677-0.736j -0.000-0.j 

Finally, let's check that the final matrix, $VQU^\dagger$, is indeed diagonal, and that its entries match the ones returned by the function.

In [7]:
# Check the diagonal entries
D = numpy.zeros((m, n), dtype=complex)
D[numpy.diag_indices(m)] = diagonal
print("V * Q * U^\dagger matches the returned diagonal:")
print(numpy.all(numpy.isclose(D, V.dot(Q.dot(U.T.conj())))))

V * Q * U^\dagger matches the returned diagonal:
True
